In [10]:
def read_data(file_path):
    with open(file_path, 'r') as f:
        path = f.read().splitlines()
    return path


train_data = read_data('train_data.txt')
val_data = read_data('val_data.txt')

In [11]:
train_data

['data/20-50/train\\35\\163892.jpg',
 'data/20-50/train\\43\\137807.jpg',
 'data/20-50/train\\37\\174673.jpg',
 'data/20-50/train\\29\\173118.jpg',
 'data/20-50/train\\36\\159522.jpg',
 'data/20-50/train\\21\\153412.jpg',
 'data/20-50/train\\29\\158739.jpg',
 'data/20-50/train\\39\\157937.jpg',
 'data/20-50/train\\49\\150977.jpg',
 'data/20-50/train\\32\\167695.jpg',
 'data/20-50/train\\26\\148888.jpg',
 'data/20-50/train\\36\\167666.jpg',
 'data/20-50/train\\32\\151209.jpg',
 'data/20-50/train\\23\\157141.jpg',
 'data/20-50/train\\30\\175537.jpg',
 'data/20-50/train\\44\\169274.jpg',
 'data/20-50/train\\30\\172298.jpg',
 'data/20-50/train\\49\\124702.jpg',
 'data/20-50/train\\45\\163197.jpg',
 'data/20-50/train\\49\\126843.jpg',
 'data/20-50/train\\50\\110168.jpg',
 'data/20-50/train\\24\\159414.jpg',
 'data/20-50/train\\29\\174117.jpg',
 'data/20-50/train\\30\\152893.jpg',
 'data/20-50/train\\44\\151788.jpg',
 'data/20-50/train\\25\\151509.jpg',
 'data/20-50/train\\49\\139499.jpg',
 

In [12]:
import os
import cv2
IMG_RESOLUTION = (224, 224)

def my_data_generator_train():
  for path in train_data:
    image = cv2.imread(path)
    image_resized = cv2.resize(image, IMG_RESOLUTION)
    image_normalized = image_resized / 255.0
    image_flattened = image_normalized.flatten()
    sample = image_flattened
    parts = path.split('\\')
    label = parts[-2]
    yield sample, (label,)

In [32]:
def my_data_generator_val():
  for path in val_data:

    image = cv2.imread(path)
    image_resized = cv2.resize(image, IMG_RESOLUTION)
    image_normalized = image_resized / 255.0
    image_flattened = image_normalized.flatten()
    sample = image_flattened
    parts = path.split('\\')
    label = parts[-2]
    yield sample, (label,)

In [33]:
my_data_generator_val()

<generator object my_data_generator_val at 0x000001EA34A67DE0>

In [34]:
import tensorflow as tf

batch_size = 32
x_shape = (IMG_RESOLUTION[0] * IMG_RESOLUTION[1] * 3,)
x_type = tf.float32
y_shape = (1,)
y_type = tf.int32

train_ds = tf.data.Dataset.from_generator(my_data_generator_train, output_signature=(
         tf.TensorSpec(shape=x_shape, dtype=x_type),
         tf.TensorSpec(shape=y_shape, dtype=y_type)))

train_ds = train_ds.shuffle(100)
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(my_data_generator_val, output_signature=(
         tf.TensorSpec(shape=x_shape, dtype=x_type),
         tf.TensorSpec(shape=y_shape, dtype=y_type)))

val_ds = val_ds.batch(batch_size)

In [41]:
x_shape

(150528,)

In [44]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping

model = models.Sequential([
    tf.keras.layers.Input(shape=x_shape),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(51, activation='softmax')
])

In [45]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [49]:
model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[early_stopping])

Epoch 1/50
    836/Unknown 1056s 1s/step - accuracy: 0.0372 - loss: 9.8449

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


836/836 ━━━━━━━━━━━━━━━━━━━━ 1171s 1s/step - accuracy: 0.0372 - loss: 9.8406 - val_accuracy: 0.0438 - val_loss: 4.6088
Epoch 2/50
836/836 ━━━━━━━━━━━━━━━━━━━━ 1058s 1s/step - accuracy: 0.0414 - loss: 4.4733 - val_accuracy: 0.0438 - val_loss: 4.0921
Epoch 3/50
836/836 ━━━━━━━━━━━━━━━━━━━━ 852s 1s/step - accuracy: 0.0412 - loss: 4.0259 - val_accuracy: 0.0438 - val_loss: 3.8035
Epoch 4/50
836/836 ━━━━━━━━━━━━━━━━━━━━ 1380s 2s/step - accuracy: 0.0415 - loss: 3.7707 - val_accuracy: 0.0438 - val_loss: 3.6300
Epoch 5/50
836/836 ━━━━━━━━━━━━━━━━━━━━ 1456s 2s/step - accuracy: 0.0387 - loss: 3.6191 - val_accuracy: 0.0435 - val_loss: 3.5322
Epoch 6/50
836/836 ━━━━━━━━━━━━━━━━━━━━ 1380s 2s/step - accuracy: 0.0403 - loss: 3.5296 - val_accuracy: 0.0438 - val_loss: 3.4703
Epoch 7/50
836/836 ━━━━━━━━━━━━━━━━━━━━ 916s 1s/step - accuracy: 0.0400 - loss: 3.4771 - val_accuracy: 0.0438 - val_loss: 3.4388
Epoch 8/50
836/836 ━━━━━━━━━━━━━━━━━━━━ 864s 1s/step - accuracy: 0.0400 - loss: 3.4503 - val_accuracy: 